The NFL is a multi-billion dollar business. There are millions of fans around the world, possibily even yourself, that invest a significant amount of time and resources to follow and watch their favorite teams and players. The teams are always working to fufill their duty to the fans. With the money they receive, they work on providing fans with the best experience possible, whether that's through giving them a great experience at the game with stadium rennovations or through building a winning team that makes fans want to tune in more. Every year, teams get around 225 million dollars to spend on players, with the money being paid through contracts. It's pretty straightforward, teams spend money on players that they think will benefit the team, and all is good, right? Obviously the answer is no, and there is a lot of nuance and subtleties that go into how much to pay players. Time and time again teams either give way too much money to certain players which ends up hurting the team, and many times teams avoid signing a certain player who ends up being a bargain for another. Then there is the question of how do we weigh which positions are worth more money. Each position has vastly different roles on a team, and therefore some are paid better than others. This however, also depends on the scheme the team has as that can affect how much they value each position. Then there are many more factors such as age and past performance that have to be accounted for. Then after all of that, even if done perfectly to the book can still end up not working. A player could not be a hard worker or lazy or had a fluke year, all of which are very hard to account for. So which are the factors you can account for? This is the question we will answer today, by looking through both basic box score predictions as well as ProFootballFocus' grading system, which breaks down the tape for every play of every player. This brings an element of context to the stats, which also can be deceiving based on what happens in a play, for example a quarterback can throw a perfect pass that gets dropped which would be considered an incompletion. We will do analysis on all different types of stats, advanced and basic, to determine which ones correlate with salary the best or how much each one correlates. This will give us a better picture of how we can predict what teams should pay a player based on their past performance and how much value in general they should add to a team. We will break down each category individually, by passing, rushing, receiving, blocking, run defense, pass rush and coverage, which will give us a nice split of positions and will allow us to differentiate the value of certain skills.

In [206]:
import pandas as pd
import numpy as np

# get salary data 
tables = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/limit-2000/')

In [207]:
import re


salary_df = tables[0].rename(columns={'Player': 'player'})
players = salary_df['player']
pl = []
start_years = []
end_years = []

for player in players:
    try:
        # split up player column and extract start year end year and name
        groups = re.search(r'(.*)  .* \| (\d{4})-(\d{4}) \(FA: (\d{4})\)', player)
        pl.append(groups.groups()[0])
        start_years.append(groups.groups()[1])
        end_years.append(groups.groups()[2])
    # if regex fails dont throw error
    except:
        pl.append(None)
        start_years.append(None)
        end_years.append(None)

# insert clean data
salary_df['player'] = pl
salary_df['start_year'] = start_years
salary_df['end_year'] = end_years
salary_df

,Rank,player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,1,Patrick Mahomes,24,10,"$450,000,000","$45,000,000","$10,000,000","$63,081,905","$141,481,905",2020,2031
1,2,Lamar Jackson,26,5,"$260,000,000","$52,000,000","$72,500,000","$112,500,000","$185,000,000",2023,2027
2,3,Josh Allen,25,6,"$258,034,000","$43,005,667","$16,500,000","$100,038,596","$150,000,000",2021,2028
3,4,Jalen Hurts,24,5,"$255,000,000","$51,000,000","$23,294,000","$110,000,000","$179,399,000",2023,2028
4,5,Russell Wilson,33,5,"$242,588,236","$48,517,647","$50,000,000","$124,000,000","$161,000,000",2022,2028
...,...,...,...,...,...,...,...,...,...,...,...
1995,1983,Jon Bostic,31,1,"$1,120,000","$1,120,000",NaN,NaN,NaN,2022,2022
1996,1983,Logan Ryan,31,1,"$1,120,000","$1,120,000",NaN,NaN,NaN,2022,2022
1997,1983,Matt Overton,36,1,"$1,120,000","$1,120,000",NaN,NaN,NaN,2022,2022
1998,1983,Jesper Horsted,26,1,"$1,120,000","$1,120,000","$30,000","$30,000","$30,000",2023,2023


In [208]:
# passing players and stats

passing_dfs = []

# passing

# 2022
passing_dfs.append(pd.read_csv('./passing/passing_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
passing_dfs.append(pd.read_csv('./passing/passing_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
passing_dfs.append(pd.read_csv('./passing/passing_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
passing_dfs.append(pd.read_csv('./passing/passing_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
passing_dfs.append(pd.read_csv('./passing/passing_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
passing_dfs.append(pd.read_csv('./passing/passing_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


passing_dfs
passing_dfs[0].columns

# merge years together
passing_df = pd.merge(passing_dfs[0], passing_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
passing_df = pd.merge(passing_df, passing_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
passing_df = pd.merge(passing_df, passing_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
passing_df = pd.merge(passing_df, passing_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
passing_df = pd.merge(passing_df, passing_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
passing_df = passing_df[passing_df['position'] == 'QB']
passing_df = passing_df[passing_df['attempts_2022'] + passing_df['attempts_2021'] + 
                        passing_df['attempts_2020'] + passing_df['attempts_2019'] + 
                        passing_df['attempts_2018'] + passing_df['attempts_2017'] > 100]

# merge players with salary
passing_df = pd.merge(passing_df, salary_df, on='player')

passing_df

,player,player_id,position,team_name_2022,player_game_count_2022,accuracy_percent_2022,aimed_passes_2022,attempts_2022,avg_depth_of_target_2022,avg_time_to_throw_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Patrick Mahomes,11765,QB,KC,20.0,78.0,685.0,747.0,7.5,2.85,...,1,24,10,"$450,000,000","$45,000,000","$10,000,000","$63,081,905","$141,481,905",2020,2031
1,Justin Herbert,28237,QB,LAC,18.0,78.8,673.0,743.0,6.9,2.74,...,242,22,4,"$26,578,755","$6,644,689","$16,890,004","$26,578,755","$26,578,755",2020,2024
2,Joe Burrow,28022,QB,CIN,19.0,77.3,660.0,715.0,7.6,2.50,...,186,23,4,"$36,190,137","$9,047,534","$23,880,100","$36,190,137","$36,190,137",2020,2024
3,Kirk Cousins,7102,QB,MIN,18.0,76.7,621.0,682.0,7.9,2.69,...,191,33,1,"$35,000,000","$35,000,000","$25,000,000","$35,000,000","$35,000,000",2022,2023
4,Trevor Lawrence,77632,QB,JAX,19.0,76.3,624.0,670.0,7.9,2.50,...,183,21,4,"$36,793,488","$9,198,372","$24,118,900","$27,598,900","$36,793,488",2021,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Case Keenum,7323,QB,BUF,2.0,28.6,7.0,7.0,8.3,1.87,...,630,35,2,"$6,250,000","$3,125,000","$750,000","$4,000,000","$4,000,000",2023,2024
59,Brian Hoyer,5277,QB,NE,1.0,83.3,6.0,6.0,4.2,1.97,...,825,37,2,"$4,500,000","$2,250,000","$875,000","$4,205,000","$4,205,000",2023,2024
60,Taysom Hill,12112,QB,0,0.0,0.0,0.0,0.0,0.0,0.00,...,164,31,4,"$40,000,000","$10,000,000",NaN,"$10,100,000","$21,500,000",2021,2025
61,Drew Lock,39517,QB,0,0.0,0.0,0.0,0.0,0.0,0.00,...,961,26,1,"$4,000,000","$4,000,000","$1,750,000","$1,750,000","$1,750,000",2023,2023


In [209]:
# rushing players and stats

rushing_dfs = []

# 2022
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


rushing_dfs
rushing_dfs[0].columns

# merge years together
rushing_df = pd.merge(rushing_dfs[0], rushing_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
rushing_df = pd.merge(rushing_df, rushing_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
rushing_df = pd.merge(rushing_df, rushing_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
rushing_df = pd.merge(rushing_df, rushing_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
rushing_df = pd.merge(rushing_df, rushing_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
rushing_df = rushing_df[rushing_df['position'] == 'HB']
rushing_df = rushing_df[rushing_df['attempts_2022'] + rushing_df['attempts_2021'] + 
                        rushing_df['attempts_2020'] + rushing_df['attempts_2019'] + 
                        rushing_df['attempts_2018'] + rushing_df['attempts_2017'] > 100]

# merge players with salary
rushing_df = pd.merge(rushing_df, salary_df, on='player')

rushing_df

,player,player_id,position,team_name_2022,player_game_count_2022,attempts_2022,avoided_tackles_2022,breakaway_attempts_2022,breakaway_percent_2022,breakaway_yards_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Derrick Henry,10679,HB,TEN,16.0,349.0,69.0,15.0,27.6,424.0,...,142,26,4,"$50,000,000","$12,500,000","$12,000,000","$25,500,000","$25,500,000",2020,2023
1,Josh Jacobs,45953,HB,LV,17.0,339.0,90.0,15.0,23.4,386.0,...,484,25,1,"$10,091,000","$10,091,000",NaN,NaN,NaN,2023,2023
2,Saquon Barkley,45791,HB,NYG,18.0,313.0,41.0,21.0,37.3,532.0,...,484,26,1,"$10,091,000","$10,091,000",NaN,NaN,NaN,2023,2023
3,Nick Chubb,45783,HB,CLV,17.0,302.0,83.0,23.0,34.6,527.0,...,185,25,3,"$36,600,000","$12,200,000","$12,000,000","$17,133,059","$20,000,000",2021,2024
4,Miles Sanders,40555,HB,PHI,20.0,294.0,52.0,13.0,21.6,306.0,...,247,25,4,"$25,400,000","$6,350,000","$5,900,000","$11,000,000","$13,000,000",2023,2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Ke'Shawn Vaughn,45866,HB,TB,7.0,17.0,3.0,0.0,0.0,0.0,...,802,23,4,"$4,755,997","$1,188,999","$1,018,908","$1,018,908","$1,018,908",2020,2023
66,David Johnson,9519,HB,NO,5.0,12.0,0.0,0.0,0.0,0.0,...,1983,30,1,"$1,120,000","$1,120,000",NaN,NaN,NaN,2022,2022
67,Myles Gaskin,45914,HB,MIA,2.0,10.0,3.0,0.0,0.0,0.0,...,1867,26,1,"$1,232,500","$1,232,500",NaN,NaN,NaN,2023,2023
68,Ameer Abdullah,9487,HB,LV,15.0,4.0,0.0,0.0,0.0,0.0,...,1720,29,1,"$1,750,000","$1,750,000","$200,000","$500,000","$500,000",2023,2023


In [210]:
# receiving players and stats

receiving_dfs = []

# 2022
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


receiving_dfs
receiving_dfs[0].columns

# merge years together
receiving_df = pd.merge(receiving_dfs[0], receiving_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
receiving_df = pd.merge(receiving_df, receiving_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
receiving_df = pd.merge(receiving_df, receiving_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
receiving_df = pd.merge(receiving_df, receiving_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
receiving_df = pd.merge(receiving_df, receiving_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
receiving_df = receiving_df[receiving_df['targets_2022'] + receiving_df['targets_2021'] + 
                        receiving_df['targets_2020'] + receiving_df['targets_2019'] + 
                        receiving_df['targets_2018'] + receiving_df['targets_2017'] > 100]

# merge players with salary
receiving_df = pd.merge(receiving_df, salary_df, on='player')

receiving_df

,player,player_id,position,team_name_2022,player_game_count_2022,avg_depth_of_target_2022,avoided_tackles_2022,caught_percent_2022,contested_catch_rate_2022,contested_receptions_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Justin Jefferson,61398,WR,MIN,18.0,10.7,11.0,73.0,55.0,22.0,...,418,21,4,"$13,122,805","$3,280,701","$7,103,856","$13,122,805","$13,122,805",2020,2024
1,Travis Kelce,7844,TE,KC,20.0,7.5,25.0,75.3,45.5,10.0,...,115,30,4,"$57,250,000","$14,312,500",NaN,"$20,750,000","$22,750,000",2020,2025
2,Tyreek Hill,10799,WR,MIA,18.0,12.6,12.0,69.6,50.0,13.0,...,19,28,4,"$120,000,000","$30,000,000","$25,500,000","$52,535,000","$72,200,000",2022,2026
3,Davante Adams,8688,WR,LV,17.0,12.8,16.0,59.5,44.1,15.0,...,14,29,5,"$140,000,000","$28,000,000","$19,250,000","$22,750,000","$65,670,000",2022,2026
4,CeeDee Lamb,61570,WR,DAL,19.0,10.3,16.0,72.5,46.7,14.0,...,393,21,4,"$14,010,012","$3,502,503","$7,749,100","$14,010,012","$14,010,012",2020,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Jalen Guyton,34164,WR,LAC,3.0,28.3,0.0,50.0,50.0,1.0,...,1867,25,1,"$1,232,500","$1,232,500","$76,000","$76,000","$76,000",2023,2023
175,Tim Patrick,12087,WR,0,0.0,0.0,0.0,0.0,0.0,0.0,...,202,27,3,"$34,000,000","$11,333,333","$3,000,000","$11,500,000","$18,500,000",2021,2024
176,Calvin Ridley,48262,WR,0,0.0,0.0,0.0,0.0,0.0,0.0,...,479,23,4,"$10,900,711","$2,725,178","$6,007,790","$8,454,251","$8,454,251",2018,2023
177,Rashard Higgins,10806,WR,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1933,27,1,"$1,187,500","$1,187,500","$152,500","$152,500","$152,500",2022,2022


In [211]:
# offense_blocking players and stats

offense_blocking_dfs = []

# 2022
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


offense_blocking_dfs
offense_blocking_dfs[0].columns

# merge years together
offense_blocking_df = pd.merge(offense_blocking_dfs[0], offense_blocking_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
offense_blocking_df = offense_blocking_df[(offense_blocking_df['position'] == 'C') | 
                                          (offense_blocking_df['position'] == 'G') | 
                                          (offense_blocking_df['position'] == 'T')]
offense_blocking_df = offense_blocking_df[offense_blocking_df['snap_counts_block_2022'] + offense_blocking_df['snap_counts_block_2021'] + 
                        offense_blocking_df['snap_counts_block_2020'] + offense_blocking_df['snap_counts_block_2019'] + 
                        offense_blocking_df['snap_counts_block_2018'] + offense_blocking_df['snap_counts_block_2017'] > 100]

# merge players with salary
offense_blocking_df = pd.merge(offense_blocking_df, salary_df, on='player')

offense_blocking_df

,player,player_id,position,team_name_2022,player_game_count_2022,block_percent_2022,declined_penalties_2022,franchise_id_2022,grades_offense_2022,grades_pass_block_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Jason Kelce,6343,C,PHI,20.0,100.0,1.0,24.0,89.4,83.3,...,386,35,1,"$14,250,000","$14,250,000","$10,335,000","$14,250,000","$14,250,000",2023,2023
1,Isaac Seumalo,10713,G,PHI,20.0,100.0,3.0,24.0,72.7,78.2,...,256,29,3,"$24,000,000","$8,000,000","$6,950,000","$6,950,000","$6,950,000",2023,2025
2,Creed Humphrey,59996,C,KC,20.0,100.0,1.0,16.0,89.9,81.3,...,670,21,4,"$5,565,208","$1,391,302","$1,407,424","$2,320,388","$2,320,388",2021,2024
3,Orlando Brown Jr.,46227,T,KC,20.0,100.0,1.0,16.0,75.4,76.8,...,93,26,4,"$64,092,000","$16,023,000","$31,100,000","$31,100,000","$31,100,000",2023,2026
4,Cordell Volson,30614,G,CIN,19.0,100.0,1.0,7.0,53.7,52.7,...,873,23,4,"$4,316,884","$1,079,221","$656,884","$656,884","$656,884",2022,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,Trystan Colon-Castillo,41117,C,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1757,24,1,"$1,660,000","$1,660,000","$250,000","$250,000","$250,000",2023,2023
317,Chuma Edoga,46232,T,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1867,25,1,"$1,232,500","$1,232,500","$152,500","$1,092,500","$1,092,500",2023,2023
318,Aaron Stinnie,28653,G,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1791,28,1,"$1,400,000","$1,400,000",NaN,"$150,000","$150,000",2023,2023
319,Mekhi Becton,59818,T,0,0.0,0.0,0.0,0.0,0.0,0.0,...,327,21,4,"$18,446,048","$4,611,512","$10,975,308","$18,446,048","$18,446,048",2020,2024


For the defense we need to more organization by position

In [212]:
# run_defense players and stats

run_defense_dfs = []

# 2022
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


run_defense_dfs
run_defense_dfs[0].columns

# merge years together
run_defense_df = pd.merge(run_defense_dfs[0], run_defense_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
run_defense_df = run_defense_df[run_defense_df['snap_counts_run_2022'] + run_defense_df['snap_counts_run_2021'] + 
                        run_defense_df['snap_counts_run_2020'] + run_defense_df['snap_counts_run_2019'] + 
                        run_defense_df['snap_counts_run_2018'] + run_defense_df['snap_counts_run_2017'] > 100]

# merge players with salary
run_defense_df = pd.merge(run_defense_df, salary_df, on='player')

run_defense_df

# pass_rush players and stats

pass_rush_dfs = []

# 2022
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


pass_rush_dfs
pass_rush_dfs[0].columns

# merge years together
pass_rush_df = pd.merge(pass_rush_dfs[0], pass_rush_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
pass_rush_df = pass_rush_df[pass_rush_df['snap_counts_pass_rush_2022'] + pass_rush_df['snap_counts_pass_rush_2021'] + 
                        pass_rush_df['snap_counts_pass_rush_2020'] + pass_rush_df['snap_counts_pass_rush_2019'] + 
                        pass_rush_df['snap_counts_pass_rush_2018'] + pass_rush_df['snap_counts_pass_rush_2017'] > 100]

# merge players with salary
pass_rush_df = pd.merge(pass_rush_df, salary_df, on='player')

# merge run defense and pass rush
front_7_df = pd.merge(pass_rush_df, run_defense_df, on=['player_id', 'player', 'position']).fillna(0)

In [213]:
# interior defensive line
idl_df = front_7_df[front_7_df['position'] == 'DI']
idl_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
0,Calais Campbell,4364,DI,BLT,15.0,2.0,1.0,3.0,71.7,12.0,...,598,36,1,"$7,000,000","$7,000,000","$3,000,000","$7,000,000","$7,000,000",2023,2023
2,Ndamukong Suh,5527,DI,PHI,11.0,0.0,0.0,24.0,55.5,3.0,...,1663,35,1,"$2,000,000","$2,000,000","$250,000","$250,000","$250,000",2022,2022
3,Tyson Alualu,5535,DI,PIT,17.0,1.0,0.0,25.0,58.5,2.0,...,674,33,2,"$5,500,000","$2,750,000","$1,925,000","$1,925,000","$1,925,000",2021,2022
7,Linval Joseph,5571,DI,PHI,11.0,0.0,0.0,24.0,60.2,0.0,...,1663,34,1,"$2,000,000","$2,000,000","$250,000","$250,000","$250,000",2022,2022
12,Cameron Heyward,6183,DI,PIT,17.0,2.0,0.0,25.0,78.5,10.0,...,91,31,4,"$65,600,000","$16,400,000","$17,500,000","$20,250,000","$26,250,000",2020,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,Sheldon Day,10737,DI,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1955,28,1,"$1,165,000","$1,165,000",0,0,0,2023,2023
370,Vincent Taylor,11949,DI,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1933,28,1,"$1,187,500","$1,187,500","$152,500","$275,000","$275,000",2022,2022
371,Maurice Hurst,38563,DI,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1864,27,1,"$1,242,000","$1,242,000",0,0,0,2023,2023
373,Levi Onwuzurike,43679,DI,0,0.0,0.0,0.0,0.0,0.0,0.0,...,554,23,4,"$8,148,893","$2,037,223","$3,286,468","$4,316,872","$4,316,872",2021,2024


In [214]:
# edge rushers
edge_df = front_7_df[front_7_df['position'] == 'ED']
edge_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
1,Calais Campbell,4364,ED,0,0.0,0.0,0.0,0.0,0.0,0.0,...,598,36,1,"$7,000,000","$7,000,000","$3,000,000","$7,000,000","$7,000,000",2023,2023
4,Brandon Graham,5538,ED,PHI,20.0,1.0,0.0,24.0,89.8,4.0,...,744,34,1,"$5,000,000","$5,000,000","$3,835,000","$5,000,000","$5,000,000",2023,2023
5,Jason Pierre-Paul,5540,ED,BLT,15.0,4.0,1.0,3.0,56.0,0.0,...,1799,33,1,"$1,350,000","$1,350,000","$150,000","$1,150,000","$1,150,000",2022,2022
6,Jerry Hughes,5556,ED,HST,17.0,0.0,2.0,13.0,71.3,0.0,...,487,33,2,"$10,000,000","$5,000,000","$2,500,000","$4,500,000","$4,500,000",2022,2023
8,Carlos Dunlap,5579,ED,KC,20.0,8.0,0.0,16.0,61.0,11.0,...,1249,33,1,"$3,000,000","$3,000,000","$1,880,000","$3,000,000","$3,000,000",2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Kayvon Thibodeaux,98444,ED,NYG,16.0,5.0,0.0,21.0,67.3,12.0,...,217,21,4,"$31,339,038","$7,834,760","$19,972,028","$31,339,038","$31,339,038",2022,2026
361,Travon Walker,98940,ED,JAX,17.0,0.0,3.0,15.0,58.6,8.0,...,180,21,4,"$37,372,621","$9,343,155","$24,360,088","$37,372,621","$37,372,621",2022,2026
362,Sam Williams,99106,ED,DAL,17.0,0.0,2.0,9.0,70.9,7.0,...,633,23,4,"$6,224,251","$1,556,063","$1,706,728","$2,694,649","$2,694,649",2022,2025
369,Stephen Weatherly,10861,ED,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1930,28,1,"$1,200,000","$1,200,000","$150,000","$650,000","$650,000",2022,2022


In [215]:
# linebackers
lb_df = front_7_df[front_7_df['position'] == 'LB']
lb_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
18,Bobby Wagner,7050,LB,LA,17.0,0.0,0.0,26.0,79.6,3.0,...,674,32,1,"$5,500,000","$5,500,000","$1,250,000","$5,500,000","$5,500,000",2023,2023
19,Lavonte David,7060,LB,TB,18.0,0.0,0.0,30.0,65.0,1.0,...,825,33,1,"$4,500,000","$4,500,000","$3,335,000","$3,335,000","$3,335,000",2023,2023
21,Demario Davis,7079,LB,NO,17.0,0.0,0.0,20.0,85.9,2.0,...,240,31,3,"$27,000,000","$9,000,000","$11,000,000","$18,350,000","$18,350,000",2020,2024
24,Jon Bostic,7831,LB,WAS,9.0,0.0,0.0,32.0,50.6,0.0,...,1983,31,1,"$1,120,000","$1,120,000",0,0,0,2022,2022
28,A.J. Klein,7930,LB,BUF,5.0,0.0,0.0,4.0,60.7,0.0,...,1802,31,1,"$1,317,500","$1,317,500","$100,000","$100,000","$100,000",2023,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,Zaven Collins,77190,LB,ARZ,16.0,2.0,1.0,1.0,65.5,2.0,...,380,21,4,"$14,690,257","$3,672,564","$8,043,824","$14,690,257","$14,690,257",2021,2025
343,Nick Bolton,81322,LB,KC,20.0,1.0,0.0,16.0,61.6,5.0,...,657,20,4,"$5,834,032","$1,458,508","$1,602,932","$2,528,115","$2,528,115",2021,2024
345,Micah Parsons,81360,LB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,345,22,4,"$17,079,793","$4,269,948","$9,781,668","$17,079,793","$17,079,793",2021,2025
367,Neville Hewitt,9836,LB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1682,28,1,"$1,950,000","$1,950,000","$350,000","$850,000","$850,000",2022,2022


In [216]:
# defense_coverage players and stats

defense_coverage_dfs = []

# 2022
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


defense_coverage_dfs
defense_coverage_dfs[0].columns

# merge years together
defense_coverage_df = pd.merge(defense_coverage_dfs[0], defense_coverage_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(0)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(0)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(0)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(0)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(0)

# remove outliers
defense_coverage_df = defense_coverage_df[defense_coverage_df['snap_counts_coverage_2022'] + defense_coverage_df['snap_counts_coverage_2021'] + 
                        defense_coverage_df['snap_counts_coverage_2020'] + defense_coverage_df['snap_counts_coverage_2019'] + 
                        defense_coverage_df['snap_counts_coverage_2018'] + defense_coverage_df['snap_counts_coverage_2017'] > 100]

# merge players with salary
defense_coverage_df = pd.merge(defense_coverage_df, salary_df, on='player')

defense_coverage_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Jason Pierre-Paul,5540,ED,BLT,15.0,0.0,2.4,100.0,4.9,3.0,...,1799,33,1,"$1,350,000","$1,350,000","$150,000","$1,150,000","$1,150,000",2022,2022
1,Kareem Jackson,5545,S,DEN,17.0,5.0,8.0,69.0,99.7,22.9,...,1663,33,1,"$2,000,000","$2,000,000","$700,000","$2,000,000","$2,000,000",2022,2022
2,Kareem Jackson,5545,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1663,33,1,"$2,000,000","$2,000,000","$700,000","$2,000,000","$2,000,000",2022,2022
3,Jerry Hughes,5556,ED,HST,17.0,0.0,0.0,0.0,0.8,0.0,...,487,33,2,"$10,000,000","$5,000,000","$2,500,000","$4,500,000","$4,500,000",2022,2023
4,Carlos Dunlap,5579,ED,KC,20.0,1.0,8.5,100.0,4.5,10.5,...,1249,33,1,"$3,000,000","$3,000,000","$1,880,000","$3,000,000","$3,000,000",2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,Marcus Allen,49263,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1610,25,1,"$2,540,000","$2,540,000",NaN,NaN,NaN,2022,2022
443,Cody Davis,8458,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1650,33,1,"$2,200,000","$2,200,000",NaN,"$500,000","$500,000",2023,2023
444,Jamal Agnew,11920,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,386,25,3,"$14,250,000","$4,750,000","$3,500,000","$4,000,000","$4,000,000",2021,2023
445,Leonard Johnson,7385,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1304,24,3,"$2,695,000","$898,333",NaN,NaN,NaN,2023,2025


In [217]:
# linebackers
full_lb_df = pd.merge(lb_df, defense_coverage_df[defense_coverage_df['position'] == 'LB'], on=['player_id', 'player', 'position'])
full_lb_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Bobby Wagner,7050,LB,LA,17.0,0.0,0.0,26.0,79.6,3.0,...,674,32,1,"$5,500,000","$5,500,000","$1,250,000","$5,500,000","$5,500,000",2023,2023
1,Lavonte David,7060,LB,TB,18.0,0.0,0.0,30.0,65.0,1.0,...,825,33,1,"$4,500,000","$4,500,000","$3,335,000","$3,335,000","$3,335,000",2023,2023
2,Demario Davis,7079,LB,NO,17.0,0.0,0.0,20.0,85.9,2.0,...,240,31,3,"$27,000,000","$9,000,000","$11,000,000","$18,350,000","$18,350,000",2020,2024
3,Jon Bostic,7831,LB,WAS,9.0,0.0,0.0,32.0,50.6,0.0,...,1983,31,1,"$1,120,000","$1,120,000",NaN,NaN,NaN,2022,2022
4,A.J. Klein,7930,LB,BUF,5.0,0.0,0.0,4.0,60.7,0.0,...,1802,31,1,"$1,317,500","$1,317,500","$100,000","$100,000","$100,000",2023,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Zaven Collins,77190,LB,ARZ,16.0,2.0,1.0,1.0,65.5,2.0,...,380,21,4,"$14,690,257","$3,672,564","$8,043,824","$14,690,257","$14,690,257",2021,2025
74,Nick Bolton,81322,LB,KC,20.0,1.0,0.0,16.0,61.6,5.0,...,657,20,4,"$5,834,032","$1,458,508","$1,602,932","$2,528,115","$2,528,115",2021,2024
75,Micah Parsons,81360,LB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,345,22,4,"$17,079,793","$4,269,948","$9,781,668","$17,079,793","$17,079,793",2021,2025
76,Neville Hewitt,9836,LB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1682,28,1,"$1,950,000","$1,950,000","$350,000","$850,000","$850,000",2022,2022


In [218]:
# cornerbacks
cb_df = defense_coverage_df[defense_coverage_df['position'] == 'CB']
cb_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
2,Kareem Jackson,5545,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1663,33,1,"$2,000,000","$2,000,000","$700,000","$2,000,000","$2,000,000",2022,2022
6,Patrick Peterson,6157,CB,MIN,18.0,2.0,12.1,60.0,99.3,14.6,...,394,32,2,"$14,000,000","$7,000,000","$5,850,000","$5,850,000","$5,850,000",2023,2024
10,Stephon Gilmore,7016,CB,IND,16.0,2.0,9.3,63.4,100.0,10.9,...,308,31,2,"$20,000,000","$10,000,000","$4,000,000","$9,510,000","$14,000,000",2022,2023
19,Justin Bethel,7654,CB,MIA,5.0,0.0,8.7,60.0,100.0,7.6,...,1802,32,1,"$1,317,500","$1,317,500","$75,000","$75,000","$75,000",2023,2023
20,Darius Slay,7817,CB,PHI,20.0,4.0,11.6,56.6,100.0,14.6,...,160,31,3,"$42,000,000","$14,000,000","$10,185,000","$24,500,000","$24,500,000",2023,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,Chris Jones,49339,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,58,26,4,"$80,000,000","$20,000,000",NaN,"$37,626,000","$60,000,000",2020,2023
435,Isaac Yiadom,49355,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1867,27,1,"$1,232,500","$1,232,500",NaN,NaN,NaN,2023,2023
438,Ifeatu Melifonwu,56203,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,791,22,4,"$4,785,852","$1,196,463","$840,620","$840,620","$840,620",2021,2024
444,Jamal Agnew,11920,CB,0,0.0,0.0,0.0,0.0,0.0,0.0,...,386,25,3,"$14,250,000","$4,750,000","$3,500,000","$4,000,000","$4,000,000",2021,2023


In [219]:
# Safeties
s_df = defense_coverage_df[defense_coverage_df['position'] == 'S']
s_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
1,Kareem Jackson,5545,S,DEN,17.0,5.0,8.0,69.0,99.7,22.9,...,1663,33,1,"$2,000,000","$2,000,000","$700,000","$2,000,000","$2,000,000",2022,2022
16,Michael Thomas,7279,S,CIN,3.0,0.0,0.0,0.0,88.2,0.0,...,487,30,1,"$10,000,000","$10,000,000","$5,000,000","$10,000,000","$10,000,000",2023,2024
17,Michael Thomas,7279,S,CIN,3.0,0.0,0.0,0.0,88.2,0.0,...,1955,32,1,"$1,165,000","$1,165,000",NaN,NaN,NaN,2023,2023
18,Harrison Smith,7641,S,MIN,15.0,5.0,10.0,80.0,97.8,16.4,...,96,32,4,"$64,000,000","$16,000,000","$9,579,410","$14,179,410","$26,379,410",2021,2025
22,Tyrann Mathieu,7850,S,NO,17.0,5.0,5.7,67.4,96.9,20.4,...,236,29,3,"$28,300,000","$9,433,333","$9,500,000","$18,000,000","$18,000,000",2022,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Sheldrick Redwine,50458,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1651,26,2,"$2,135,000","$1,067,500",NaN,NaN,NaN,2023,2024
437,Armani Watts,51264,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1933,26,1,"$1,187,500","$1,187,500","$152,500","$402,500","$402,500",2022,2022
442,Marcus Allen,49263,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1610,25,1,"$2,540,000","$2,540,000",NaN,NaN,NaN,2022,2022
443,Cody Davis,8458,S,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1650,33,1,"$2,200,000","$2,200,000",NaN,"$500,000","$500,000",2023,2023
